In [24]:
# pip install textacy
import textacy
import spacy
import es_core_news_sm # python -m spacy download es   (no anda con internet feo)
nlp = es_core_news_sm.load(disable=['parser']) # parser is for POS

In [20]:
path = '../input/historia-de-indias-1.txt'
file = open(path, 'r')
raw_text = file.read()

In [ ]:

#doc = textacy.Doc(raw_text)
# raw_txt = textacy.io.read_text('../input/historia-de-indias-1.txt', lines=True)
# for i in raw_text:
#     print(i)
#nlp = nlp(raw_text)

textacy.Corpus('es', texts=raw_text)

In [19]:
read_text?

In [9]:
file.close()